In [35]:
import numpy as np
import pandas as pd
import os
path = os.getcwd()

In [36]:
size = '64'
path = './All_spreadsheet_'+size

In [37]:
### read evenly-spaced discretiztion of empirical de-meaned CDF of div growth
input_1 = pd.read_excel(path+'/1.xlsx',header=None)
states_empirical_CDF = input_1.iloc[:-1,0].values

### read quadrature abscissa for theta_hat
states_theta_hat = input_1.iloc[:-1,1].values

### read quadrature abscissa for theta_i,i=1...1000
def states_from_simulations(i):
    # set i from 1...1000
    return input_1.iloc[:-1,i+1].values

### read mean for div growth
div_growth_mean = input_1.iloc[-1,0]

### read KL_i,i=1...1000
KL = input_1.iloc[-1,2:].values

### read stationary distribution for markov chain approx from theta_hat
input_2 = pd.read_excel(path+'/2.xlsx',header=None)
dist_theta_hat = input_2.iloc[:,0].values

### read stationary distribution for markov chain approx from theta_i,i=1...1000
def dist_from_simulations(i):
    # set i from 1...1000
    return input_2.iloc[:,i].values

### read transition matrix for markov chain approx from theta_hat
transition_matrix_theta_hat = pd.read_excel(path+'/3.xlsx',header=None).values

### read transition matrix for markov chain approx from theta_i,i=1...1000
def trans_matrix_from_simulations(i):
    # set i from 1...1000
    return pd.read_excel(path+'/'+str(i+3)+'.xlsx',header=None).values

In [38]:
dist_from_simulations(0)

array([0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625, 0.015625, 0.015625,
       0.015625, 0.015625, 0.015625, 0.015625])

In [39]:
states_theta_hat

array([-0.22627823, -0.21271727, -0.20149318, -0.19147758, -0.18223996,
       -0.17355865, -0.1653007 , -0.15737854, -0.14973057, -0.1423114 ,
       -0.13508629, -0.12802792, -0.12111424, -0.11432711, -0.10765136,
       -0.10107411, -0.09458425, -0.08817212, -0.0818292 , -0.07554793,
       -0.06932149, -0.06314372, -0.05700899, -0.05091211, -0.04484824,
       -0.03881289, -0.03280177, -0.02681084, -0.02083622, -0.01487413,
       -0.00892094, -0.00297306,  0.00297306,  0.00892094,  0.01487413,
        0.02083622,  0.02681084,  0.03280177,  0.03881289,  0.04484824,
        0.05091211,  0.05700899,  0.06314372,  0.06932149,  0.07554793,
        0.0818292 ,  0.08817212,  0.09458425,  0.10107411,  0.10765136,
        0.11432711,  0.12111424,  0.12802792,  0.13508629,  0.1423114 ,
        0.14973057,  0.15737854,  0.1653007 ,  0.17355865,  0.18223996,
        0.19147758,  0.20149318,  0.21271727,  0.22627823])

In [40]:
def states_to_dividend_growth(states : np.array, a = 0.01037):
    dividend_growth = np.zeros((len(states),1))
    for indx, element in np.ndenumerate(states):
        dividend_growth[indx] += np.exp(a+element)
    return dividend_growth

dividend_growth = states_to_dividend_growth(states_theta_hat)

In [41]:
def growth_to_H(dividend_growth, b=0.03630):
    b_matrix = np.array([[np.exp(b), np.exp(-b)],
                         [np.exp(b), np.exp(-b)]])
    one = np.ones((1,len(dividend_growth)))
    H = np.kron(np.matmul(dividend_growth,one),
                    b_matrix)
    return H

H = growth_to_H(dividend_growth)

In [42]:
def sdf_equation(state):
    return np.exp(-0.8970 + (1.2038*state))

def states_to_sdf(states : np.array, sdf_equation : callable):
    sdf = np.zeros((len(states),1))
    for indx, element in np.ndenumerate(states):
        sdf[indx] += sdf_equation(element)
    return sdf

sdf = states_to_sdf(states_theta_hat, sdf_equation)

In [43]:
def parameters_to_xi(gamma, function, b=0.03630):

    if function == 'CRRA':
        return -b*gamma
    if function == 'IES':
        return  0.0412 - (0.0775*gamma)
    else:
        raise Exception("Function must be either 'CRRA' or 'IES'")
        

gamma=2
function = 'CRRA'
xi = parameters_to_xi(gamma, function, b=0.03630)

In [44]:
def sdf_to_M(sdf: np.array, xi):
    xi_matrix = np.array([[np.exp(xi), np.exp(-xi)],
                         [np.exp(xi), np.exp(-xi)]])
    
    one = np.ones((1,len(dividend_growth)))
    M = np.kron(np.matmul(sdf,one),
                    xi_matrix)
    
    return M

M = sdf_to_M(sdf, xi)

In [45]:
M.shape

(128, 128)

In [46]:
def Psi_and_Pi_to_B(Psi, Pi):
    size = Psi.shape[0]
    B = np.zeros((size,size))
    for row in range(size):
        for pi, psi in zip(Pi[row], Psi[row]):
            B[row][row] += pi*psi
            
        B[row][row] /= size
    return B

Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
Psi = np.multiply(H, M)
B = Psi_and_Pi_to_B(Psi, Pi)

In [47]:
def H_M_and_Pi_to_A(H, M, Pi):

    I = np.identity(H.shape[0])
    A = np.multiply(H, M)
    A = np.multiply(A, Pi)
    A = I - A
    return A

A = H_M_and_Pi_to_A(H, M, Pi)

In [48]:
C = np.matmul(np.linalg.inv(B),A)

In [49]:
unit = np.ones((C.shape[0],1))
unit /= C.shape[0]

v_128 = np.matmul(np.linalg.inv(C), unit)
v_128.sort()

In [50]:
d_half = states_empirical_CDF + div_growth_mean
d_half.shape

(64,)

In [51]:
def cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean):

    d_half = states_empirical_CDF + div_growth_mean

    size = d_half.shape[0]
    d_full = np.zeros((2*size,1))

    for i, observation in enumerate(d_half):
        index = 2*i
        d_full[index] = observation
        if i == size-1:
            d_full[index+1] = observation
        else:
            ave = observation + d_half[i+1]
            ave /= 2
            d_full[index+1] = ave
            
    return d_full

d_full = cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean)

In [52]:
d_full

array([[-0.28428393],
       [-0.2802594 ],
       [-0.27623487],
       [-0.27221034],
       [-0.26818581],
       [-0.26416128],
       [-0.26013675],
       [-0.25611222],
       [-0.25208769],
       [-0.24806316],
       [-0.24403863],
       [-0.2400141 ],
       [-0.23598958],
       [-0.23196505],
       [-0.22794052],
       [-0.22391599],
       [-0.21989146],
       [-0.21586693],
       [-0.2118424 ],
       [-0.20781787],
       [-0.20379334],
       [-0.19976881],
       [-0.19574428],
       [-0.19171975],
       [-0.18769522],
       [-0.18367069],
       [-0.17964616],
       [-0.17562164],
       [-0.17159711],
       [-0.16757258],
       [-0.16354805],
       [-0.15952352],
       [-0.15549899],
       [-0.15147446],
       [-0.14744993],
       [-0.1434254 ],
       [-0.13940087],
       [-0.13537634],
       [-0.13135181],
       [-0.12732728],
       [-0.12330275],
       [-0.11927823],
       [-0.1152537 ],
       [-0.11122917],
       [-0.10720464],
       [-0

In [53]:
def C(gamma, utility_function):
    
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    return C

In [54]:
folder = './C_matrix_'+size+'/'

for utility_function in ['CRRA','IES']:
    for gamma in [2,5,10]:
        matrix = C(gamma, utility_function)
        df = pd.DataFrame(matrix)
        matrix_name = utility_function+'_'+str(gamma)
        df.to_excel(folder+matrix_name+'.xlsx',header=None,index=False)
        
        test_df = pd.read_excel(folder+matrix_name+'.xlsx', header=None)
        test = test_df.to_numpy()
        


In [55]:
def model_growth(gamma, utility_function):
    
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    
    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)
    
    d = cdf_and_mean_to_d(states_empirical_CDF, div_growth_mean)

    solution = d-v
    return np.matmul(solution.T, solution)

In [56]:
for utility_function in ['CRRA','IES']:
    for gamma in [2,5,10]:
        growth = model_growth(gamma, utility_function)
        print('utility_function = ',utility_function,', gamma = ',gamma,', growth = ',growth)

utility_function =  CRRA , gamma =  2 , growth =  [[2.93106132]]
utility_function =  CRRA , gamma =  5 , growth =  [[2.93098105]]
utility_function =  CRRA , gamma =  10 , growth =  [[2.93060774]]
utility_function =  IES , gamma =  2 , growth =  [[2.93104247]]
utility_function =  IES , gamma =  5 , growth =  [[2.93065645]]
utility_function =  IES , gamma =  10 , growth =  [[2.92829209]]


In [57]:
def HHL_parameters(gamma, utility_function):
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H(dividend_growth)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xi = parameters_to_xi(gamma, utility_function, b=0.03630)
    M = sdf_to_M(sdf, xi)
    
    Pi = np.kron(transition_matrix_theta_hat, 
            np.array([[1/2, 1/2],[1/2, 1/2]]))
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    
    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)

    return C, unit, v

In [58]:
gamma = 2
utility_function = 'IES'
C, unit, v = HHL_parameters(gamma, utility_function)
#np.linalg.eigvals(C_128)

In [59]:
def growth_to_H_sv(dividend_growth, pi_G, gamma_G, b=0.03630):

    bb = b*(1-(gamma_G*pi_G))/(1-pi_G)
    bg = b*gamma_G
    b_matrix = np.array([[np.exp(bg), np.exp(-bg)],
                         [np.exp(bb), np.exp(-bb)]])
    one = np.ones((1,len(dividend_growth)))
    H = np.kron(np.matmul(dividend_growth,one),
                    b_matrix)
    return H

In [60]:
def sdf_to_M_sv(sdf: np.array, xi_g, xi_b):
    xi_matrix = np.array([[np.exp(xi_g), np.exp(-xi_b)],
                         [np.exp(xi_g), np.exp(-xi_b)]])
    
    one = np.ones((1,len(dividend_growth)))
    M = np.kron(np.matmul(sdf,one),
                    xi_matrix)
    
    return M

In [61]:
def parameters_to_xi_sv(gamma, function, pi_G, gamma_G, b=0.03630):

    bb = b*(1-(gamma_G*pi_G))/(1-pi_G)
    bg = b*gamma_G

    if function == 'CRRA':
        return (-bg*gamma, bb*gamma)
    if function == 'IES':
        xi_neutral = 0.0412 - (0.0775*gamma) - (b*gamma)
        return (xi_neutral+bg*gamma,  xi_neutral+bb*gamma)
    else:
        raise Exception("Function must be either 'CRRA' or 'IES'")

In [62]:
def HHL_parameters_sv(gamma, utility_function, pi_G, gamma_G):
    dividend_growth = states_to_dividend_growth(states_theta_hat)
    H = growth_to_H_sv(dividend_growth, pi_G=pi_G, gamma_G=gamma_G)
    sdf = states_to_sdf(states_theta_hat, sdf_equation)
    xig, xib = parameters_to_xi_sv(gamma, utility_function, pi_G=pi_G, gamma_G=gamma_G, b=0.03630)
    M = sdf_to_M_sv(sdf, xig, xib)

    S = np.array([[pi_G, 1-pi_G],[pi_G, 1-pi_G]])
    Pi = np.kron(transition_matrix_theta_hat, S)
    Psi = np.multiply(H, M)
    B = Psi_and_Pi_to_B(Psi, Pi)
    
    A = H_M_and_Pi_to_A(H, M, Pi)
    
    C = np.matmul(np.linalg.inv(B),A)
    
    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)

    return C, unit, v

In [63]:
subfolder_name = "C_matrix_"+str(size)

pi_G = 0.8
gamma_G = 0.3

for utility_function in ['CRRA', 'IES']:
    for gamma in [2, 5, 10]:
        for pi_G, gamma_G in [(0.8, 0.3), (0.95, 0.01)]:
            C, _, _ = HHL_parameters_sv(gamma, utility_function, pi_G, gamma_G)
            file_name = 'SV_'+str(gamma_G)+str(pi_G)+'_'+utility_function+'_'+str(gamma)+'.xlsx'
            file_path = os.path.join(subfolder_name, file_name)
            c_df = pd.DataFrame(C)
            c_df.to_excel(file_path, header=False, index=False)

In [64]:
#rd_transition_matrix = np.loadtxt('rare_disasters_transition_matrix_'+str(2*int(size))+'.txt', delimiter=',')
#rd_hgrid = np.loadtxt('rare_disasters_h_grid_'+str(2*int(size))+'.txt', delimiter=',')

rd_transition_matrix = np.loadtxt('rare_disasters_even/rare_disasters_transition_matrix_'+str(2*int(size))+'.txt', delimiter=',')
rd_hgrid = np.loadtxt('rare_disasters_even/rare_disasters_h_grid_'+str(2*int(size))+'.txt', delimiter=',')

In [65]:
print(size)
print(rd_hgrid)
print(rd_transition_matrix)
#print(np.diag(1+rd_hgrid))

64
[-0.1260546  -0.12598325 -0.12585493 -0.12566969 -0.12542763 -0.1251289
 -0.12477368 -0.12436217 -0.12389464 -0.12337134 -0.12279261 -0.12215878
 -0.12147023 -0.12072737 -0.11993065 -0.11908055 -0.11817757 -0.11722225
 -0.11621516 -0.11515691 -0.11404812 -0.11288947 -0.11168163 -0.11042534
 -0.10912135 -0.10777043 -0.10637339 -0.10493107 -0.10344433 -0.10191405
 -0.10034116 -0.09872659 -0.09707131 -0.0953763  -0.09364258 -0.09187119
 -0.09006319 -0.08821965 -0.08634167 -0.08443039 -0.08248694 -0.08051248
 -0.07850819 -0.07647528 -0.07441495 -0.07232844 -0.070217   -0.06808188
 -0.06592436 -0.06374574 -0.06154732 -0.0593304  -0.05709631 -0.0548464
 -0.052582   -0.05030446 -0.04801516 -0.04571545 -0.04340671 -0.04109032
 -0.03876767 -0.03644014 -0.03410913 -0.03177602 -0.02944222 -0.02710912
 -0.0247781  -0.02245057 -0.02012792 -0.01781154 -0.0155028  -0.01320309
 -0.01091378 -0.00863625 -0.00637185 -0.00412193 -0.00188785  0.00032907
  0.0025275   0.00470612  0.00686364  0.00899875  

In [66]:
rho = 0.0657
g_D = 0.025

def HHL_parameters_RD(gamma, utility_function):
    
    Pi = rd_transition_matrix
    diag = np.diag(1+rd_hgrid)
    #Pi = np.kron(rd_transition_matrix, 
    #        np.array([[1/2, 1/2],[1/2, 1/2]]))
    #diag = np.kron(np.diag(1+rd_hgrid), np.array([0.5,0.5]))
    scalar = np.exp(-rho+g_D)
    id = np.identity(Pi.shape[0])

    C = np.matmul(diag, Pi)
    C *= scalar
    C = id - C
    

    unit = np.ones((C.shape[0],1))
    unit /= np.linalg.norm(unit)
    
    v = np.matmul(np.linalg.inv(C),
                      unit)

    return C, unit, v

HHL_parameters_RD(2, 'IES')

(array([[ 9.99999854e-01, -9.09015512e-08, -3.88798689e-08, ...,
         -2.85281923e-09, -6.64648991e-09, -1.06426135e-08],
        [-1.00224362e-10,  1.00000000e+00, -1.35747760e-10, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.11629320e-10, -1.32893888e-10,  1.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [0.08838835],
        [

In [67]:
size

'64'

In [68]:
subfolder_name = "C_matrix_"+str(size)

for utility_function in ['CRRA']:
    for gamma in [2]:
        C, _, _ = HHL_parameters_RD(gamma, utility_function)
        file_name = 'RD_'+utility_function+'_'+str(gamma)+'.xlsx'
        file_path = os.path.join(subfolder_name, file_name)
        c_df = pd.DataFrame(C)
        c_df.to_excel(file_path, header=False, index=False)